In [6]:
import pandas as pd
data = pd.read_csv('data_merged.csv', index_col=0)

from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
target = "Inflation_Rate"
features = list(train_set.columns)
features = [x for x in features if x not in ['Date', "Inflation_Rate"]]

X_train = train_set[features]
y_train = train_set[[target]]
X_test = test_set[features]
y_test = test_set[[target]]
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
poly.fit(X_train)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

In [7]:
print("Original number of features: "+str(len(features)))
print("Final number of features: "+str(X_train.shape[1]))



Original number of features: 7
Final number of features: 36


In [3]:
import numpy as np
array_of_features = np.array(features)
feature_names_transformed = poly.get_feature_names_out(array_of_features)
feature_names_transformed

array(['1', 'Unnamed: 0', 'Prime_Rate', 'Crude_prices', '10_year_bond',
       'manu_labor_rate', 'CAD_M1_MoneySupply', 'USD_CAD_rate',
       'SNP_TSX_Close', 'Unnamed: 0^2', 'Unnamed: 0 Prime_Rate',
       'Unnamed: 0 Crude_prices', 'Unnamed: 0 10_year_bond',
       'Unnamed: 0 manu_labor_rate', 'Unnamed: 0 CAD_M1_MoneySupply',
       'Unnamed: 0 USD_CAD_rate', 'Unnamed: 0 SNP_TSX_Close',
       'Prime_Rate^2', 'Prime_Rate Crude_prices',
       'Prime_Rate 10_year_bond', 'Prime_Rate manu_labor_rate',
       'Prime_Rate CAD_M1_MoneySupply', 'Prime_Rate USD_CAD_rate',
       'Prime_Rate SNP_TSX_Close', 'Crude_prices^2',
       'Crude_prices 10_year_bond', 'Crude_prices manu_labor_rate',
       'Crude_prices CAD_M1_MoneySupply', 'Crude_prices USD_CAD_rate',
       'Crude_prices SNP_TSX_Close', '10_year_bond^2',
       '10_year_bond manu_labor_rate', '10_year_bond CAD_M1_MoneySupply',
       '10_year_bond USD_CAD_rate', '10_year_bond SNP_TSX_Close',
       'manu_labor_rate^2', 'manu_labo